<a href="https://www.kaggle.com/code/dongjj/dataloader?scriptVersionId=107021751" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [19]:
from scipy.io import loadmat
import numpy as np
from PIL import Image
import cv2
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import defaultdict
from datetime import datetime
import torch.optim as optim
from torch.optim import lr_scheduler
import pandas as pd
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True
Image.MAX_IMAGE_PIXELS = None
Image_path='../input/backgroundremovedversion/*'
label_path='../input/mayo-clinic-strip-ai/train.csv'
trans_preprocess = transforms.Compose([
    transforms.Resize([512,512]),
])
for i in glob.glob('../input/backgroundremovedversion/*'):
    
    for j in glob.glob(f'{i}/*'):
        img=Image.open(j)
        img.load()
        img=trans_preprocess(img)
        img_data=img.getdata()
        img=np.array(img_data)
        split=j.split('/')[-1]
        p=f'./{split}'
        cv2.imwrite(p,img)
             
  

In [20]:
!cd /kaggle/working
!tar -czvf my_work.zip -C . .

./
./79d1c1_1-0.png
./9d609a_0-0.png
./b7f78e_0-0.png
./3ad0bb_0-0.png
./eebf8d_1-1.png
./3152c0_0-0.png
./b7f78e_0-1.png
./d862da_0-1.png
./9f7649_0-1.png
./d4c955_0-0.png
./9fd56b_0-0.png
./3d10be_1-1.png
./1d5335_0-1.png
./6747be_0-0.png
./ae0487_0-0.png
./251bfc_0-0.png
./03e6b7_0-0.png
./5cae18_1-0.png
./e7bec8_0-0.png
./616c62_0-1.png
./c30f14_1-0.png
./a2b566_0-1.png
./6f6e0c_0-0.png
./35b75a_0-1.png
./83bfad_0-1.png
./75e090_0-0.png
./c92b51_0-0.png
./4d15a9_0-0.png
./673181_0-1.png
./af7a93_0-0.png
./437435_0-1.png
./508589_0-0.png
./d8db68_0-0.png
./e3f976_0-0.png
./abc4a3_2-0.png
./6e211d_0-1.png
./0ee750_1-1.png
./d4c955_1-0.png
./fbf2eb_0-0.png
./b5aece_0-1.png
./280c26_1-0.png
./e72352_0-1.png
./31adaa_0-0.png
./777311_1-1.png
./e5bd0b_0-1.png
./5d4b88_1-1.png
./b6fffe_0-0.png
./aeb3bd_0-0.png
./588e11_0-0.png
./c65db4_1-1.png
./b6fffe_0-1.png
./5f520a_0-0.png
./f4c9ff_0-1.png
./9d71ef_0-0.png
./15de51_0-0.png
./9fdc31_0-0.png
./250944_0-0.png
./f8712b_0-0.png
./0ff6fc_0-

In [15]:
trans_preprocess = transforms.Compose([
    transforms.Resize([512,512]),
])
img=Image.open('../input/backgroundremovedversion/output-part1-2/280c26_1-0.png')
img.load()
img=trans_preprocess(img)
img_data=img.getdata()
img=np.array(img_data)
cv2.imwrite('./280c26_1-0.png',img)

True

In [2]:
df=pd.read_csv(label_path)


In [3]:
# trans = transforms.Compose([
#     transforms.Resize([512,512]),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize([0.790595  , 0.67119867, 0.8091853 ], [0.05493367, 0.05985045, 0.04747279]) 
# ])

# class Dataset(Dataset):
#         def __init__(self, Image_path, label_path, transform):
#             df=pd.read_csv(label_path)
#             a=df['image_id'].tolist()
#             image_ls=[]
#             isin_list=[]
#             for i in glob.glob(Image_path):
#                 for j in glob.glob(f'{i}/*'):
#                     l=[]
#                     l.append(j)
#                     split=j.split('/')[-1]
#                     output=split.split('-')[0]
#                     l.append(output)
#                     image_ls.append(l)
#                     isin_list.append(output)
#             sorted(image_ls,key=lambda s: s[1])     
#             sorted(isin_list)
#             label_ls=[]
#             for i in isin_list:
#                 if i in a:
#                     label_ls.append(df[df['image_id']==i]['label'])
#             self.img_ls=image_ls
#             self.label_ls=label_ls
        
#         def __len__(self):
#             return len(self.img_ls)
#         def __getitem__(self, idx):
#             img_name = self.img_ls[idx]
#             img=Image.open(img_name[0])
#             img.load()
#             img = self.transform(img)
#             label = self.label_ls[idx]
#             return img,label

In [4]:
image_ls=[]
isin_list=[]
label_ls=[]
label_list=[]
to_check=df['image_id'].tolist()
for i in glob.glob('../input/backgroundremovedversion/*'):
    
    for j in glob.glob(f'{i}/*'):
        l=[]
        l.append(j)
        split=j.split('/')[-1]
        output=split.split('-')[0]
        l.append(output)
        image_ls.append(l)
        isin_list.append(output)
image_ls=sorted(image_ls,key=lambda s: s[1])     
isin_list=sorted(isin_list) 
for a in isin_list:
    label_ls.extend(df[df['image_id']==a]['label'].tolist())
for i in label_ls:
    if i =="CE":
        label_list.append(0)
    else:
        label_list.append(1) 
label_list=torch.Tensor(label_list)
image_list=[]
for i in image_ls:
    image_list.append(i[0])
trans_train = transforms.Compose([
    transforms.Resize([512,512]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406 ], [0.229, 0.224, 0.225]) 
])
trans_test = transforms.Compose([
    transforms.Resize([512,512]),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406 ], [0.229, 0.224, 0.225]) 
])
class Dataset(Dataset):
        def __init__(self, image_ls, label_ls,transform):
            self.img_ls=image_ls
            self.label_ls=label_ls
            self.transform=transform
        def __len__(self):
            return len(self.img_ls)
        def __getitem__(self, idx):
            img_name = self.img_ls[idx]
            img=Image.open(img_name) # from disk to cpu ram
            img.load()
            label = self.label_ls[idx]
            img = self.transform(img)
            return img,label
        
        
class D:
    def __init__(self, image_ls, labels_ls):
        self.images = [load(n) for n in image_ls] # from disk to CPU ram
        self.labels = labels_ls
        
    def __get_items(self, idx):
        return self.images[idx], self.labels[idx]

In [5]:
base_model = models.resnet18(pretrained=True)
num_features=base_model.fc.in_features
base_model.fc=nn.Linear(num_features,2)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [6]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 0.001  
batch_size = 8 
test_image_ls=image_list[615:715]
test_label_ls=label_list[615:715]
train_image_ls=image_list[0:614]
train_label_ls=label_list[0:614]
train_set = Dataset(train_image_ls,train_label_ls,transform=trans_train)
train_loader = DataLoader(train_set, batch_size=batch_size,shuffle=True)
test_set = Dataset(test_image_ls,test_label_ls,transform=trans_test)
test_loader = DataLoader(test_set, batch_size=1,shuffle=False)
print(device)

cuda


In [11]:
from torch.utils.tensorboard import SummaryWriter
model = base_model
model=model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

loss_fn = F.cross_entropy
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)
        import pdb; pdb.set_trace()
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels.long())
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(test_loader):
        vinputs, vlabels = vdata
        vinputs=vinputs.to(device)
        vlabels=vlabels.to(device)
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels.long())
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
> /tmp/ipykernel_18/3170787648.py(19)train_one_epoch()
     17         import pdb; pdb.set_trace()
     18         # Zero your gradients for every batch!
---> 19         optimizer.zero_grad()
     20 
     21         # Make predictions for this batch



ipdb>  inputs


tensor([[[[2.1462, 2.1119, 2.1119,  ..., 1.9235, 2.0605, 2.1462],
          [2.1462, 2.0434, 2.1290,  ..., 1.9407, 2.0605, 2.1290],
          [2.2147, 2.1119, 2.0434,  ..., 1.9578, 1.8550, 2.0092],
          ...,
          [2.0777, 1.7009, 1.5810,  ..., 1.9920, 1.8893, 1.9578],
          [1.7865, 1.7180, 1.4783,  ..., 1.9235, 1.8379, 1.9407],
          [1.7523, 1.7694, 1.7009,  ..., 1.8550, 1.8722, 1.8893]],

         [[1.9559, 1.9909, 2.1310,  ..., 1.5532, 1.8333, 1.9559],
          [1.8859, 1.8859, 2.1835,  ..., 1.7458, 1.8333, 2.0084],
          [1.9734, 1.9559, 2.0609,  ..., 1.8683, 1.6933, 1.7983],
          ...,
          [2.0784, 1.7108, 1.4832,  ..., 1.9734, 1.6758, 2.0259],
          [1.6583, 1.6933, 1.2206,  ..., 1.5182, 1.4832, 1.9384],
          [1.3606, 1.5707, 1.4657,  ..., 1.3256, 1.6758, 1.9034]],

         [[0.6705, 0.8448, 1.7685,  ..., 1.7337, 1.4200, 0.7228],
          [0.8099, 1.0365, 2.0474,  ..., 1.9603, 1.3502, 0.6531],
          [0.7402, 1.0539, 1.6117,  ..., 1

ipdb>  n


> /tmp/ipykernel_18/3170787648.py(22)train_one_epoch()
     20 
     21         # Make predictions for this batch
---> 22         outputs = model(inputs)
     23 
     24         # Compute the loss and its gradients



...

tensor([[-0.3001, -0.0117],
        [-0.0078, -0.2902],
        [ 0.0510, -0.0291],
        [-1.1254,  0.2579],
        [-0.1402,  0.1762],
        [ 0.3541, -0.5232],
        [-0.3894,  0.1459],
        [-0.9180,  0.1765]], device='cuda:0', grad_fn=<AddmmBackward0>)


ipdb>  labels


tensor([1., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


ipdb>  q


BdbQuit: 

In [12]:
! ls

__notebook_source__.ipynb  runs


In [15]:
x = Image.open('../input/backgroundremovedversion/output-part1-2/280c26_1-0.png')

In [17]:
np.array(x).shape

(5120, 7680, 3)